In [3]:
# data science
import scipy as sp
import numpy as np
import pandas as pd
# i/o
import json
# type hints
import ast
import pkg_resources


In [4]:
print(path_countries_per_region := '/home/weinold/github/pylcaio_integration_with_brightway/helper_functions/countries_per_region.json')
print(path_countries := '/home/weinold/github/pylcaio_integration_with_brightway/helper_functions/countries.json')

/home/weinold/github/pylcaio_integration_with_brightway/helper_functions/countries_per_region.json
/home/weinold/github/pylcaio_integration_with_brightway/helper_functions/countries.json


In [5]:
%%capture
import pymrio
print(path_home := os.path.join('/home', os.getlogin()))
print(path_dir_out := os.path.join(path_home, 'data_pylcaio_output'))
print(path_file_out_pymrio_class_instance_pickle := os.path.join(path_dir_out, 'pymrio_class_instance.pkl'))
with open(path_file_out_pymrio_class_instance_pickle, 'rb') as file_in:
    exiobase = pd.read_pickle(file_in)

⚠️ countries are IO countries, not LCA countries
!!! fix this!

In [6]:
with open(file = path_countries_per_region, mode = 'r', encoding = 'utf-8') as filestream:
    lca_countries_in_io_regions: dict = json.load(fp = filestream)
with open(file = path_countries, mode = 'r', encoding = 'utf-8') as filestream:
    lca_countries: list = json.load(fp = filestream)

### 1.1. identify rest-of-world regions (RoW)

In [7]:
lca_countries_absent_in_io_regions: dict = {
    io_region: [country for country in lca_countries if country not in lca_countries_in_io_region]
    for (io_region, lca_countries_in_io_region) in lca_countries_in_io_regions.items()
}

In [46]:
num_lca_regions: int = len(lca_countries_in_io_regions.keys())

In [47]:
io_sectors: list = list(exiobase.get_sectors())

In [15]:
listmatrixxx = []
listlisteee = []
listdfff = []
for i in range(0, len(lca_countries_absent_in_io_regions)):
    listmatrixxx.append('matrixxx' + str(i))
    listlisteee.append('listeee' + str(i))
    listdfff.append('dfff' + str(i))

In [ ]:
def identify_rows(self):
    """ Method to identify the various unique Rest of the World (RoW) regions of the LCA database

    Returns:
    --------
        The updated self.dictRoW in which unique RoW regions are identified in terms of countries they include

    """

    # contains a list of activityNameId's that are 'PRO_f.io_geography[i]=='RoW (=LCA RoW)'
    unique_activities_using_row = list(
        set( # removes duplicates
            self.PRO_f.activityNameId[
                [i for i in self.PRO_f.index if self.PRO_f.io_geography[i] == 'RoW']].tolist()
        )
    )

    RoW_activities = defaultdict(list)

    # how and WHERE are `geography` and `io_geography` matched?

    tupl = [ # list of tuples (activity, geography) that have io_geography==RoW !!!???
        i for i in zip( # zip creates tuple
            self.PRO_f.activityNameId.loc[ # pandas series with column activityNameId of all those activities that are 'PRO_f.io_geography[i]=='RoW (=LCA RoW)
                [
                    i for i in self.PRO_f.index
                    if self.PRO_f.activityNameId[i] in unique_activities_using_row
                ]
            ],
            self.PRO_f.io_geography.loc[ # WTF, why even do this, if above you only get those activities that already have self.PRO_f.io_geography[i] == 'RoW']
                [
                    i for i in self.PRO_f.index
                    if self.PRO_f.activityNameId[i] in unique_activities_using_row
                ]
            ]
        )
    ]

    # ok, dictionary is filled
    for activity, geography in tupl:
        RoW_activities[activity].append(geography)


    # remove RoW
    RoW_activities = {activity: [geography1 for geography1 in geography if geography1 != 'RoW'] for activity, geography in RoW_activities.items()}

    # delete from RoW_activities processes which had only RoW as geography and are thus empty now
    for key in [i for i in list(RoW_activities.keys()) if RoW_activities[i] == []]:
        del RoW_activities[key]
        
    # put every element to the same level (elements that are lists are transformed to lists of lists)
    for values in list(RoW_activities.values()):
        for i in range(0, len(values)):
            if values[i] in self.countries_per_regions.keys():
                values[i] = self.countries_per_regions[values[i]]
    # for elements that are lists of lists stemming from the replacement of ['RER'] by [['AT','BE',...]],
    # add all of the together in a single list
    for keys in RoW_activities.keys():
        for item in RoW_activities[keys]:
            if isinstance(item, list):
                RoW_activities[keys] = sum_elements_list(RoW_activities[keys])
    # remove duplicates inside the elements
    for keys in list(RoW_activities.keys()):
        RoW_activities[keys] = list(set(RoW_activities[keys]))
    # need to sort to identify duplicates whose elements would be ordered differently and thus be treated as not
    # duplicated
    for keys in RoW_activities.keys():
        RoW_activities[keys].sort()
    # identify the combination of countries that are NOT inside the residual of each process
    dictactrow = {}
    residual_geo_IO_to_remove = ['WA', 'WE', 'WF', 'WL', 'WM']
    for keys in RoW_activities.keys():
        dictactrow[keys] = list(set(self.listcountry) - set(RoW_activities[keys]) - set(residual_geo_IO_to_remove))
    unique_RoWs = []
    for keys in dictactrow.keys():
        if dictactrow[keys] not in unique_RoWs:
            unique_RoWs.append(dictactrow[keys])
    # create name for the values of the different RoW
    listname = []
    for i in range(0, len(unique_RoWs)):
        listname.append('RoW' + '(' + str(i) + ')')
    # put all of that in dictRoW
    for i in range(0, len(unique_RoWs)):
        self.dictRoW[listname[i]] = unique_RoWs[i]
    try:
        # if RoWs are empty because processes from ecoinvent are too described
        del [[k for k in self.dictRoW.keys() if len(self.dictRoW[k]) == 0][0]]
    except IndexError:
        pass
    for keys in dictactrow:
        for keys2 in self.dictRoW:
            if dictactrow[keys] == self.dictRoW[keys2]:
                dictactrow[keys] = keys2
    RoW_matrix = pd.DataFrame(list(dictactrow.values()), index=list(dictactrow.keys()), columns=['RoW_geography'])
    self.PRO_f = self.PRO_f.merge(RoW_matrix, left_on='activityNameId', right_on=RoW_matrix.index, how='outer')
    self.PRO_f.index = self.PRO_f.activityId + '_' + self.PRO_f.productId
    self.PRO_f = self.PRO_f.reindex(self.processes_in_order)
    self.PRO_f.io_geography.update(self.PRO_f.RoW_geography[self.PRO_f.io_geography == 'RoW'])
    self.PRO_f = self.PRO_f.drop('RoW_geography', axis=1)
    # might be some RoW or empty lists left in PRO_f
    self.PRO_f.io_geography[self.PRO_f.io_geography == 'RoW'] = 'GLO'
    self.PRO_f.io_geography.loc[[i for i in self.PRO_f.index if type(self.PRO_f.io_geography[i]) == list]] = 'GLO'

In [ ]:
total_prod_country =  pd.DataFrame(
    self.X_io.todense(),
    index=pd.MultiIndex.from_product(
        [self.regions_of_IO, self.sectors_of_IO],
        names=['region', 'sector']
    ),
    columns=['production']
)

In [11]:
number_of_products_IO = len([i for i in exiobase.get_sectors()])

In [12]:
total_prod_country = exiobase.x

In [13]:
exiobase.get_sectors()

Index(['Paddy rice', 'Wheat', 'Cereal grains nec', 'Vegetables, fruit, nuts',
       'Oil seeds', 'Sugar cane, sugar beet', 'Plant-based fibers',
       'Crops nec', 'Cattle', 'Pigs',
       ...
       'Paper for treatment: landfill',
       'Plastic waste for treatment: landfill',
       'Inert/metal/hazardous waste for treatment: landfill',
       'Textiles waste for treatment: landfill',
       'Wood waste for treatment: landfill',
       'Membership organisation services n.e.c. (91)',
       'Recreational, cultural and sporting services (92)',
       'Other services (93)', 'Private households with employed persons (95)',
       'Extra-territorial organizations and bodies'],
      dtype='object', name='sector', length=200)

In [ ]:
listmatrixxxx = []
listlisteeee = []
listdffff = []

for k in range(0, len(list(self.dictRoW.keys()))):
    listmatrixxxx.append('matrixxxx' + str(k))
    listlisteeee.append('listeeee' + str(k))
    listdffff.append('dfff' + str(k))
    listdrop = []
    
    for i in range(0, len(self.dictRoW)):
        listadd = []
        for j in range(0, len(io_regions)):
            if io_regions[j] not in list(dictRoW.values())[i]:
                listadd.append(self.listcountry[j])
        listdrop.append(listadd)

for i in range(0, len(list(self.dictRoW.keys()))):
    listadd = []
    listmatrixxxx[i] = self.total_prod_country.drop(listdrop[i], axis=0, level=0)
    for k in range(0, self.number_of_products_IO):
        somme = 0
        for j in range(0, len(listmatrixxxx[i]), self.number_of_products_IO):
            somme += listmatrixxxx[i].iloc[j + k, 0]
        listadd.append(somme)
    listlisteeee[i] = listadd
    listdffff[i] = pd.DataFrame(listlisteeee[i], listact, [list(self.dictRoW.keys())[i]])
    self.total_prod_RoW = self.total_prod_RoW.join(listdffff[i], how='outer')